<a href="https://colab.research.google.com/github/EWHA-AI24-Project/Weather-to-Wildfire-Prediction/blob/main/notebooks/model_2/model2_tuning_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-optimize

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
from sklearn.metrics import make_scorer, average_precision_score, roc_auc_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 로드
data = pd.read_csv('Data2_train.csv')
X = data.drop(columns=['FIRE_START_DAY'])
y = data['FIRE_START_DAY']

# 트레인/검증 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# y_train, y_val을 명시적으로 int형으로 변환 (0, 1)
y_train = y_train.astype(int)
y_val = y_val.astype(int)

# 커스텀 점수 함수
def custom_score(y_true, y_pred_proba, **kwargs):
    pr_auc = average_precision_score(y_true, y_pred_proba)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    return 0.5 * pr_auc + 0.5 * roc_auc

# Scorer 객체로 변환
custom_scorer = make_scorer(custom_score, needs_proba=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 8.0 MB/s eta 0:00:00


In [ ]:
# 하이퍼파라미터 공간

param_space = {
    'n_estimators': Integer(100, 300),
    'max_depth': Integer(5, 50),
    'min_samples_split': Integer(2, 20),
    'min_samples_leaf': Integer(1, 10),
    'max_features': Categorical(['sqrt', 'log2']),
    'bootstrap': Categorical([True, False])
}

# === 베이지안 최적화 ===

rf = RandomForestClassifier(random_state=42)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

bayes_search = BayesSearchCV(
    estimator=rf,
    search_spaces=param_space,
    scoring=custom_scorer,
    cv=cv,
    n_iter=150,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

bayes_search.fit(X_train, y_train)

# 결과 출력

print("✅ Best CV Score (Custom):", bayes_search.best_score_)
print("🔧 Best Parameters:", bayes_search.best_params_)

# 검증셋 성능 확인
best_model = bayes_search.best_estimator_
y_val_proba = best_model.predict_proba(X_val)[:, 1]

pr_auc = average_precision_score(y_val, y_val_proba)
roc_auc = roc_auc_score(y_val, y_val_proba)
custom = 0.5 * pr_auc + 0.5 * roc_auc

print("📌 Validation PR AUC :", pr_auc)
print("📌 Validation ROC AUC:", roc_auc)
print("🎯 Custom Score (0.5 * PR AUC + 0.5 * ROC AUC):", custom)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi